In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import median_absolute_error, accuracy_score,roc_auc_score
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


/home/user/conda/lib/python3.9/site-packages/lightautoml/transformers/text.py:22: UserWarning: 'gensim' - package isn't installed
  warnings.warn("'gensim' - package isn't installed")


In [3]:
# utils

def map_class(x, task, reader):
    if task.name == 'multiclass':
        return reader[x]
    else:
        return x

mapped = np.vectorize(map_class)

def score(task, y_true, y_pred):
    if task.name == 'binary':
        return roc_auc_score(y_true, y_pred)
    elif task.name == 'multiclass':
        return accuracy_score(y_true, np.argmax(y_pred, 1))
    elif task.name == 'reg' or task.name == 'multi:reg':
        return median_absolute_error(y_true, y_pred)
    else:
        raise 'Task is not correct.'
        
def take_pred_from_task(pred, task):
    if task.name == 'binary' or task.name == 'reg':
        return pred[:, 0]
    elif task.name == 'multiclass' or task.name == 'multi:reg':
        return pred
    else:
        raise 'Task is not correct.'
        
def use_plr(USE_PLR):
    if USE_PLR:
        return "plr"
    else:
        return "cont"

### 0.2 Constants

Here we setup the constants to use in the kernel:
- `N_THREADS` - number of vCPUs for LightAutoML model creation
- `N_FOLDS` - number of folds in LightAutoML inner CV
- `RANDOM_STATE` - random seed for better reproducibility
- `TEST_SIZE` - houldout data part size 
- `TIMEOUT` - limit in seconds for model to train
- `TARGET_NAME` - target column name in dataset
- `TASK` - task name, 'reg', 'binary', 'multiclass', 'multi:reg'
- `ALGOS_FOR_BLEND` - algorithms used in blending
- `USE_PLR` - if True use PLR embedder for continuous features, else Basic Embedder
- `TRAIN_BS` - train batch size

In [4]:
RANDOM_STATE = 42
N_THREADS = 16

In [5]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

### 0.3 Data loading

In [6]:
train = pd.read_csv('../input/playground-series-s4e7/train.csv')
test = pd.read_csv('../input/playground-series-s4e7/test.csv')

In [ ]:
#X_train, X_val = train_test_split(train, test_size=0.2, random_state=42, shuffle=True, stratify=train.Response)

In [16]:
task = Task('binary') 
automl = TabularAutoML(
    task = task, 
    timeout = 600 * 3600,
    cpu_limit = 16,
    general_params = {"use_algos": [['resnet']]}, # ['nn', 'mlp', 'dense', 'denselight', 'resnet', 'snn', 'node', 'autoint', 'fttransformer'] or custom torch model
    nn_params = {
        "n_epochs": 10, 
        "bs": 1024, 
        "num_workers": 0, 
        "path_to_save": None, 
        "freeze_defaults": True,
        "cont_embedder": 'plr',
        'cat_embedder': 'weighted',
        "hidden_size": 32,
        'hid_factor': [3, 3],
        'block_config': [3, 3],
        'embedding_size': 32,
        'stop_by_metric': True,
        'verbose_bar': True,
        "snap_params": { 'k': 2, 'early_stopping': True, 'patience': 1, 'swa': True }
    },
    nn_pipeline_params = {"use_qnt": False, "use_te": False},
    reader_params = {'n_jobs': 16, 'cv': 5, 'random_state': 42, 'advanced_roles': True}
)

out_of_fold_predictions = automl.fit_predict(
    train, #valid_data=X_val,
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

[23:28:33] Stdout logging level is DEBUG.
[23:28:33] Task: binary

[23:28:33] Start automl preset with listed constraints:
[23:28:33] - time: 2160000.00 seconds
[23:28:33] - CPU: 16 cores
[23:28:33] - memory: 16 GB

[23:28:33] Train data shape: (11504798, 12)

[23:29:50] Feats was rejected during automatic roles guess: []
[23:29:51] Layer 1 train process start. Time left 2159921.88 secs
[23:30:17] number of text features: 0 
[23:30:17] number of categorical features: 8 
[23:30:17] number of continuous features: 2 
[23:30:17] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_resnet_0 ...
[23:30:17] Training params: {'num_workers': 0, 'pin_memory': False, 'max_length': 256, 'is_snap': False, 'input_bn': False, 'max_emb_size': 256, 'bert_name': None, 'pooling': 'cls', 'device': device(type='cuda', index=0), 'use_cont': True, 'use_cat': True, 'use_text': False, 'lang': 'en', 'deterministic': True, 'multigpu': False, 'random_state': 42, 'model': 'resnet', 'model_with_emb': False, 'path_to_save': Non

val: 100%|██████████| 2248/2248 [00:18<00:00, 119.14it/s]


[23:33:00] Epoch: 0, train loss: 0.24678844213485718, val loss: 0.24346543848514557, val metric: 0.8911400061785146


val: 100%|██████████| 2248/2248 [00:19<00:00, 114.66it/s]


[23:35:43] Epoch: 1, train loss: 0.241507887840271, val loss: 0.24279046058654785, val metric: 0.8919753141525005


...

[00:22:52] Epoch: 2, train loss: 0.23834727704524994, val loss: 0.24314239621162415, val metric: 0.8918363504152836


val: 100%|██████████| 2248/2248 [00:19<00:00, 117.53it/s]


[00:25:32] Epoch: 3, train loss: 0.23495908081531525, val loss: 0.24437564611434937, val metric: 0.8906467689043878


val: 100%|██████████| 2248/2248 [00:19<00:00, 117.40it/s]


[00:25:52] Early stopping: val loss: 0.2423781007528305, val metric: 0.8926402886498171
[00:25:58] Fitting Lvl_0_Pipe_0_Mod_0_TorchNN_resnet_0 finished. score = 0.8920865559193433
[00:25:58] Lvl_0_Pipe_0_Mod_0_TorchNN_resnet_0 fitting and predicting completed
[00:25:58] Time left 2156555.13 secs

[00:25:58] Layer 1 training completed.

[00:25:58] Automl preset training completed in 3444.97 seconds

[00:25:58] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (5 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN_resnet_0) 



In [17]:
roc_auc_score(train.Response, out_of_fold_predictions.data)

0.8920865559193433

In [18]:
pred = automl.predict(test)

test: 100%|██████████| 7491/7491 [00:43<00:00, 172.84it/s]


In [19]:
import joblib
joblib.dump((out_of_fold_predictions.data[:, 0], pred.data[:, 0]), 'resnet_5fold_oof_test_089208.jbl')
joblib.dump(automl, 'resnet_5fold_model_089208.jbl')

['resnet_5fold_model_089208.jbl']